In [18]:
import numpy as np
import math

In [19]:
class CLCG:
    def __init__(self, K):
        self.K = K
        self.A = [0.0 for i in range(K)]
        self.m  = [0.0 for i in range(K)]
        self.length = 0
        self.seed = [1 for i in range(K)]
        self.c = [0.0 for i in range(K)]

    def element_wise_multiply(self, vector1, vector2):
        result = [vector1[i] * vector2[i] for i in range(len(vector1))]
        return result
    
    def element_wise_remainder(self, vector1, vector2):
        result = [vector1[i] % vector2[i] for i in range(len(vector1))]
        return result
    
    def next(self,x = []):
        result = 0
        for i in range(len(x)):
            result += (((-1)**i) * x[i])
        result = result % (self.m[0] - 1)
        if result ==0:
            result = (self.m[0] - 1) / self.m[0]
        else:
            result = result / self.m[0]
        return result

    def maxlength(self):
        P = 1
        for i in range(self.K):
            P = P * (self.m[i] - 1)
        P = P / pow(2, self.K - 1)
        return P
    def generate(self, length):
        X = np.empty([length, self.K])
        X[0] = self.seed
        result = np.empty(length)
        for i in range(1, length):
            
            X[i] = self.element_wise_remainder([x + y for x, y in zip(self.element_wise_multiply(X[i-1], self.A), self.c)], self.m) 

        for i in range(length):
            result[i] = self.next(X[i])
            
        return result
    def d_alpha(self, a, n):
        result = math.sqrt(((-1) / (2 * n)) * math.log(2 * a))
        return result
    
    def KS_test(self, data : np.array, a = 0.05):
        n = len(data)
        sorted_data = np.sort(data)
        ecdf = np.arange(1, n + 1) / n  # Empirical CDF
        cdf = sorted_data  # CDF of the uniform distribution in the interval [0, 1]
        d_stat = np.max(np.abs(ecdf - cdf))
        
        # Calculate critical value
        d_alpha = self.d_alpha(a , n)
        if d_alpha >= d_stat :
            print("KS Test: Fail to reject null hypothesis (data follows uniform distribution)")
            return True
        else:
            print("KS Test: Reject null hypothesis (data does not follow uniform distribution)")
            return False
    
    def autocorrelation_test(self, data, i, m):
        N = len(data)
        
        # Calculate M
        M = (N - i) // m - 1

        if M < 0:
            raise ValueError("M is negative, check the values of i and m.")

        # Calculate autocorrelation coefficient
        sum_product = sum(data[i + k * m] * data[i + (k + 1) * m] for k in range(M ))
        rho_hat = (1 / (M + 1)) * sum_product - 0.25

        # Calculate standard deviation of autocorrelation coefficient
        sigma_rho_hat = np.sqrt((13 * M + 7) / (12 * (M + 1)))

        # Calculate test statistic
        Z_0 = rho_hat / sigma_rho_hat

        print(f"Test statistic (Z_0): {Z_0}")
        if Z_0 <= 1.96 and Z_0 >= -1.96 :
            print("Failed to reject the null hypothesis. (data has independency)")
        else:
            print("The null hypothesis is rejected and data does not have independency.")
        return Z_0

    

In [20]:
ty = CLCG(2)
ty.A = [48271, 40692]
ty.m = [60013, 60017]
ty.seed = [1234, 5679]
maximum_length = ty.maxlength()
lst = ty.generate(1000)
lst

array([9.25916052e-01, 1.51633813e-01, 3.51423858e-01, 8.93139820e-03,
       6.34995751e-01, 3.25762751e-01, 4.99158516e-01, 8.90873644e-01,
       9.29998500e-01, 8.45650109e-02, 4.79329479e-01, 3.45958376e-01,
       9.65574126e-01, 7.40372919e-01, 8.62813057e-02, 2.59010548e-01,
       1.04310733e-01, 9.86402946e-01, 4.23874827e-01, 6.30779998e-01,
       3.99980004e-01, 5.10556046e-01, 4.12910536e-02, 8.51215570e-01,
       6.60590205e-01, 3.86116341e-01, 7.83930148e-01, 1.18457668e-01,
       4.81695633e-01, 7.94411211e-01, 6.66572243e-01, 8.46966491e-01,
       5.21287054e-01, 4.79062870e-01, 9.50194125e-01, 4.81062436e-02,
       4.80379251e-01, 5.96620732e-01, 9.47794644e-02, 9.69040041e-01,
       9.03504241e-01, 5.60045324e-02, 7.54669822e-02, 8.63046340e-01,
       7.90878643e-01, 1.80010998e-01, 3.78967890e-01, 7.38923233e-01,
       2.12603936e-01, 6.82618766e-01, 3.08266542e-03, 5.05790412e-01,
       4.49435956e-01, 3.47591355e-02, 5.68876743e-01, 8.87191109e-01,
      

In [21]:
KS_result = ty.KS_test(lst)

KS Test: Fail to reject null hypothesis (data follows uniform distribution)


In [22]:
Independency = ty.autocorrelation_test(lst, 0, 2)
Independency

Test statistic (Z_0): -0.016737520180669426
Failed to reject the null hypothesis. (data has independency)


-0.016737520180669426

In [23]:
maximum_length

1800840096.0